Берем отызывы за лето (из архива с материалами или предыдущего занятия)
1. Учим conv сеть для классификации - выбить auc выше 0.95
2. Предобучаем word2vec и его эмбединга инициализируем сетку, как влияет на качество?

In [1]:
from sklearn import model_selection
import nltk
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
import re
import pandas as pd
import numpy as np
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Input, Embedding, Conv1D, GlobalMaxPool1D
from keras.objectives import categorical_crossentropy
from keras.callbacks import EarlyStopping
from keras import optimizers
from gensim.models import Word2Vec
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
max_words = 1000
max_len = 100
num_classes = 3
epochs = 20
batch_size = 512
print_batch_n = 100

In [3]:
data = pd.read_excel('/content/drive/My Drive/отзывы за лето.xls')

In [4]:
exclude = set(punctuation)
stop_words = set(get_stop_words("ru"))
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in stop_words]
    return " ".join(txt)

# Делаем предобработку и сокращаем количество классов до 2
data['text'] = data['Content'].apply(preprocess_text)
data = data[data['Rating'] != 3]
data['target'] = data['Rating'] > 3
data['target'] = data['target'].astype(int)
data.head()

,Rating,Content,Date,text,target
0,5,It just works!,2017-08-14,it just works,1
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14,целое удобноной приложениеиз минус хотеть боль...,1
2,5,Отлично все,2017-08-14,отлично,1
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14,зависать 1 работа антивирус ранее пользоваться...,1
4,5,"Очень удобно, работает быстро.",2017-08-14,удобно работать быстро,1


In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data['text'], data['target'], test_size=0.2,
                                                    random_state=13, stratify=data['target'])

In [6]:
train_corpus = " ".join(X_train)
train_corpus = train_corpus.lower()
tokens = word_tokenize(train_corpus)
tokens_filtered = [word for word in tokens if word.isalnum()]

In [7]:
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]
tokens_filtered_top[:20]

['приложение',
 'удобно',
 'работать',
 'удобный',
 'отлично',
 'нравиться',
 'отличный',
 'хороший',
 'телефон',
 'супер',
 'быстро',
 'мочь',
 'обновление',
 'пользоваться',
 'банк',
 'антивирус',
 'устраивать',
 'сбербанк',
 'пароль',
 'карта']

In [8]:
len(tokens_filtered_top)

699

In [9]:

vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [10]:
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]


x_train = np.asarray([text_to_sequence(text, max_len) for text in X_train], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_len) for text in X_test], dtype=np.int32)

In [11]:
num_classes = 2
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=[keras.metrics.AUC()])

early_stopping=EarlyStopping(monitor='val_loss')  

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/20
28/28 [==============================] - 10s 357ms/step - loss: 0.4828 - auc: 0.8626 - val_loss: 0.3962 - val_auc: 0.9550
Epoch 2/20
28/28 [==============================] - 10s 348ms/step - loss: 0.3173 - auc: 0.9528 - val_loss: 0.2536 - val_auc: 0.9661
Epoch 3/20
28/28 [==============================] - 10s 346ms/step - loss: 0.2112 - auc: 0.9747 - val_loss: 0.1976 - val_auc: 0.9764
Epoch 4/20
28/28 [==============================] - 10s 346ms/step - loss: 0.1726 - auc: 0.9817 - val_loss: 0.1881 - val_auc: 0.9783
Epoch 5/20
28/28 [==============================] - 10s 348ms/step - loss: 0.1592 - auc: 0.9843 - val_loss: 0.1921 - val_auc: 0.9784


In [12]:
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)

print('Test score:', score[0])
print('Test accuracy:', score[1])

8/8 [==============================] - 1s 87ms/step - loss: 0.1755 - auc: 0.9814
Test score: 0.17552632093429565
Test accuracy: 0.9814278483390808


In [13]:
data['text_tekens'] = data['text'].apply(word_tokenize)

w2v_model = Word2Vec(data['text_tekens'], size=128, window=5, min_count=1, workers=8)
w2v_model.train(data['text_tekens'], total_examples=data['text_tekens'].shape[0], epochs=20)


DIM = w2v_model.vector_size
NUM = len(tokens_filtered_top) + 1
# Инициализируем матрицу embedding слоя нулями
embedding_matrix = np.zeros((NUM, DIM))
# Добавляем NUM наиболее часто встречающихся слов из обучающей выборки в embedding слой
for item in vocabulary.items():
    if item[1] >= NUM:
        break
    if item[0] in w2v_model.wv.vocab.keys():
        embedding_matrix[item[1]] = w2v_model.wv[item[0]]


model = Sequential()
# Сначала замораживаем веса слоя
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len, weights=[embedding_matrix], trainable=False))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [14]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=[keras.metrics.AUC()])


early_stopping=EarlyStopping(monitor='val_loss')  

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/20
28/28 [==============================] - 7s 268ms/step - loss: 0.3600 - auc_1: 0.9283 - val_loss: 0.2817 - val_auc_1: 0.9530
Epoch 2/20
28/28 [==============================] - 7s 258ms/step - loss: 0.2457 - auc_1: 0.9636 - val_loss: 0.2439 - val_auc_1: 0.9648
Epoch 3/20
28/28 [==============================] - 7s 258ms/step - loss: 0.2216 - auc_1: 0.9706 - val_loss: 0.2307 - val_auc_1: 0.9682
Epoch 4/20
28/28 [==============================] - 7s 258ms/step - loss: 0.2047 - auc_1: 0.9754 - val_loss: 0.2177 - val_auc_1: 0.9716
Epoch 5/20
28/28 [==============================] - 7s 257ms/step - loss: 0.1925 - auc_1: 0.9784 - val_loss: 0.2080 - val_auc_1: 0.9743
Epoch 6/20
28/28 [==============================] - 7s 257ms/step - loss: 0.1790 - auc_1: 0.9817 - val_loss: 0.2020 - val_auc_1: 0.9748
Epoch 7/20
28/28 [==============================] - 7s 257ms/step - loss: 0.1724 - auc_1: 0.9825 - val_loss: 0.1968 - val_auc_1: 0.9764
Epoch 8/20
28/28 [==============================

In [15]:
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)

print('Test score:', score[0])
print('Test accuracy:', score[1])

8/8 [==============================] - 1s 83ms/step - loss: 0.1891 - auc_1: 0.9779
Test score: 0.18906031548976898
Test accuracy: 0.9779139757156372


In [16]:

# Размораживаем Embedding слой и дообучаем сеть
model.layers[1].trainable = True
adam = optimizers.Adam(lr=0.0001)
model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=[keras.metrics.AUC()])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 128)          89600     
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 98, 128)           49280     
_________________________________________________________________
activation_3 (Activation)    (None, 98, 128)           0         
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1290      
_________________________________________________________________
activation_4 (Activation)    (None, 10)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                

In [17]:

early_stopping=EarlyStopping(monitor='val_loss')  

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/20
28/28 [==============================] - 8s 270ms/step - loss: 0.1538 - auc_2: 0.9861 - val_loss: 0.1948 - val_auc_2: 0.9766
Epoch 2/20
28/28 [==============================] - 7s 262ms/step - loss: 0.1515 - auc_2: 0.9865 - val_loss: 0.1938 - val_auc_2: 0.9768
Epoch 3/20
28/28 [==============================] - 7s 257ms/step - loss: 0.1500 - auc_2: 0.9869 - val_loss: 0.1940 - val_auc_2: 0.9768


In [18]:

score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

8/8 [==============================] - 1s 84ms/step - loss: 0.1844 - auc_2: 0.9789


Test score: 0.1843503713607788
Test accuracy: 0.9789301156997681


есть небольшое улучшение